In [2]:
from tkinter import *
from tkinter.ttk import Progressbar
from tkinter.ttk import Combobox
from tkinter.ttk import Notebook
from tkinter import filedialog as fd
from tkinter.messagebox import showinfo
import tkinter.font
import numpy as np
import pandas as pd 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.stats import zscore
from sklearn import decomposition
from IPython.display import display, Latex
from sklearn.cluster import KMeans
import pickle
import sklearn.cluster as cluster
from pathlib import Path
import os
import glob
from tqdm import tqdm_notebook
from lifelines.plotting import plot_lifetimes
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.statistics import logrank_test
from lifelines.statistics import multivariate_logrank_test
from lifelines.statistics import pairwise_logrank_test
import IPython.display as display
from PIL import Image

In [4]:
class Homepage():
    def __init__(self, parent):
        self.gui(parent)

    def gui(self, parent):
        if parent == 0:
            self.w1 = Tk()
            self.w1.configure(bg = '#c0d3ff')
            self.w1.geometry('500x500')
        else:
            self.w1 = Frame(parent)
            self.w1.configure(bg = '#c0d3ff')
            self.w1.place(x = 0, y = 0, width = 500, height = 500)
        self.button1 = Button(self.w1, text = "Press here to input the features image list", bg = "#ffffff", fg = "#000000", font = tkinter.font.Font(family = "Rockwell", size = 10), cursor = "arrow", state = "normal")
        self.button1.place(x = 50, y = 100, width = 400, height = 75)
        self.button1['command'] = self.Features_list
        self.button2 = Button(self.w1, text = "Press here to input the Mice Data", bg = "#ffffff", fg = "#000000", font = tkinter.font.Font(family = "Rockwell", size = 10), cursor = "arrow", state = "normal")
        self.button2.place(x = 50, y = 220, width = 400, height = 75)
        self.button2['command'] = self.Mice_data
        self.button4 = Button(self.w1, text = "Analyze data", bg = "#ffffff", fg = "#000000", font = tkinter.font.Font(family = "Rockwell", size = 10), cursor = "arrow", state = "normal")
        self.button4.place(x = 50, y = 350, width = 120, height = 40)
        self.button4['command'] = self.analyze
        self.button5 = Button(self.w1, text = "To results", bg = "#ffffff", fg = "#000000", font = tkinter.font.Font(family = "Rockwell", size = 8), cursor = "arrow", state = "normal")
        self.button5.place(x = 330, y = 350, width = 120, height = 40)
        self.button5['command'] = self.to_results
        self.ltext1 = Entry(self.w1, bg = "#ffffff", fg = "#000000", font = tkinter.font.Font(family = "Rockwell", size = 16), cursor = "arrow", state = "normal")
        self.ltext1.place(x = 170, y = 0, width = 140, height = 32)
        self.ltext1.insert(INSERT, "Pathomics 3")

    def Mice_data(self):
        self.w1.destroy()
        filetypes=(('text files','*.txt'),('All files','*.*'),('Excel files','*.xls'),('CSV files','*.csv'))
        filename=fd.askopenfilename(title='Press here to input the txt file',initialdir ='/',filetypes=filetypes)
        mice_data=pd.read_csv(str(filename))
        mice_data.to_csv(r'/Users/jacob/Desktop/Python project files/Results_GUI/mice.csv')#here will be directory for intermediate file, 
        print("File selected")
        

    def analyze(self):
        print("hello")
        
    def Features_list(self):
        self.w1.destroy()
        directory = input("Enter the path of your folder:")
        read_files = glob.glob(directory + "/*.txt")
        average_all = pd.DataFrame([])
        list1 = []
        for file in read_files:
            df = pd.read_csv(file, sep = '\t')
            image = df.iloc[0,0]
            numbers = df.iloc[:,5:]
            average = numbers.mean(axis = 0, skipna = False)
            df_average = pd. DataFrame([average])
            df_average.insert(0,'Image', image)
            average_all = pd.concat([average_all, df_average])

        
        folder_name = 'Results_GUI'
        folder = os.path.join(directory, folder_name)
        if os.path.exists(folder):
            os.makedirs(folder)
        #important to change the path in the line of code below up until results, which means (results/Summary_Table.csv')
        # should be added to the path inputed
        average_all.to_csv(r'/Users/jacob/Desktop/Python project files/Results_GUI/summary_table_GUI.csv')

    def to_results(self):
        self.w1.destroy()
        a=Analysis_page(0)
        a.w1.mainloop()

class Analysis_page():
    def __init__(self, parent):
        self.gui(parent)

    def gui(self, parent):
        if parent == 0:
            self.w1 = Tk()
            self.w1.configure(bg = '#c0d3ff')
            self.w1.geometry('520x560')
        else:
            self.w1 = Frame(parent)
            self.w1.configure(bg = '#c0d3ff')
            self.w1.place(x = 0, y = 0, width = 520, height = 560)
        self.button1 = Button(self.w1, text = "Kaplan-Meier graph", bg = "#ffffff", fg = "#080808", font = tkinter.font.Font(family = "Rockwell", size = 10), cursor = "arrow", state = "normal")
        self.button1.place(x = 160, y = 100, width = 200, height = 42)
        self.button1['command']= self.Kaplan_Meier_graph
        self.button2 = Button(self.w1, text = "Cox-analysis", bg = "#ffffff", fg = "#0b0b0b", font = tkinter.font.Font(family = "Rockwell", size = 10), cursor = "arrow", state = "normal")
        self.button2.place(x = 160, y = 180, width = 200, height = 42)
        self.button2['command']= self.Cox_analysis
        self.button3 = Button(self.w1, text = "Log-rank test analysis", bg = "#ffffff", fg = "#090909", font = tkinter.font.Font(family = "Rockwell", size = 10), cursor = "arrow", state = "normal")
        self.button3.place(x = 160, y = 260, width = 200, height = 42)
        self.button3['command']= self.Log_rank_test_analysis
        self.button4 = Button(self.w1, text = "Anova table ", bg = "#ffffff", fg = "#060606", font = tkinter.font.Font(family = "Rockwell", size = 10), cursor = "arrow", state = "normal")
        self.button4.place(x = 160, y = 340, width = 200, height = 42)
        self.button4['command']= self.Anova_table 
        self.button5 = Button(self.w1, text = "Home", bg = "#ffffff", fg = "#080808", font = tkinter.font.Font(family = "Rockwell", size = 10), cursor = "arrow", state = "normal")
        self.button5.place(x = 30, y = 490, width = 90, height = 32)
        self.button5['command']=self.Home
    def Home(self):
        self.w1.destroy()
        a=Homepage(0)
        a.w1.mainloop()
    def Kaplan_Meier_graph(self):
        print('Kaplan_Meier_graph')
    def Cox_analysis(self):
        print('Cox_analysis')
    def Log_rank_test_analysis(self):
        print('Log_rank_test_analysis')
    def Anova_table(self):
        print('Anova_table')
a = Analysis_page(0)
a.w1.mainloop()

File selected
